In [376]:
# import dependencies 
from splinter import Browser 
from bs4 import BeautifulSoup as soup

import pandas as pd
import time 

In [198]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [380]:
# visit site
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

# parse site
html = browser.html
html_soup = soup(html, 'html.parser')

# drill down with html tags that contain desired data 
mars_info_container = html_soup.select('div.description')
len(mars_info_container)
mars_info_container[0]

4

<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>

In [397]:
mars_info_container[1].find('a')

<a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a>

In [398]:
hemispheres_info = []
base_url = 'https://astrogeology.usgs.gov'

In [399]:
# loop through scraped data, scrape additonal data and format 
for i in list(range(len(mars_info_container))):
    # parse extracted data and retrieve article header 
    header = mars_info_container[i].find('h3').get_text()
    
    # retrieve embedded parital url and convert to full webpage link
    article_partial_url = mars_info_container[i].find('a').get('href')
    article_full_url = f'{base_url}{article_partial_url}'
    
    # visit article site 
    browser.visit(article_full_url)
    html = browser.html
    #parse page 
    image_parse = soup(html, 'html.parser')
    
    #retreive full-resolution image image url via tags
    image_url = image_parse.select_one('ul li a').get('href')
    
    #format data into dictionary 
    hemispheres_info.append({'img_url':image_url,'title':header})
    time.sleep(.05)

In [400]:
# review 
hemispheres_info

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [ ]:
# close scraping application 
browser.quit()

-----